In [ ]:
import zipfile
from PIL import Image
import pytesseract
import cv2 as cv
import numpy as np
# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')
# parsed_img_src = {}
small_img_list = []
with zipfile.ZipFile('readonly/small_img.zip', 'r') as files:
    
    for file in files.infolist():
        # printing out the name of the file
        display(file.filename)
        # name
        name = file.filename
        # image 
        img = Image.open(files.open(name))
        # text
        img_str = pytesseract.image_to_string(img.convert('L'))

#         if ("Christopher" in img_str):
#             my_dic = {"name":name, "img":img, "text":img_str}
#             small_img_list.append(my_dic)
        my_dic = {"name":name, "img":img, "text":img_str}
        small_img_list.append(my_dic)
# print(data_list)

import math

def extract_faces(img,para1):
    # extract the retangle of the faces
    gray = np.array(img.convert("L"))
    faces = face_cascade.detectMultiScale(gray,para1,4)
    
    # if no faces are detected
    if (len(faces) == 0):
        return None
    # extract faces into the list imgs
    faces_imgs = []
    
    for x,y,w,h in faces:
        faces_imgs.append(img.crop((x,y,x+w,y+h)))
    
    # compute nrows and ncols
    ncols = 5
    nrows = math.ceil(len(faces) / ncols)
    
    # contact sheet
    contact_sheet=Image.new(img.mode, (550, 110*nrows))
    x, y = (0, 0)
    
    for face in faces_imgs:
        face.thumbnail((110,110))
        contact_sheet.paste(face, (x,y))
        
        if x+110 == contact_sheet.width:
            x = 0
            y += 110
        else:
            x += 110
            
    return contact_sheet

for element in small_img_list:
    if 'Christopher' in element["text"]:
        print("Results found in file {}".format(element["name"]))

        img = element["img"]
        contact_sheet = extract_faces(img,1.35)
        if contact_sheet is not None:
            display(contact_sheet)
        else:
            print("But there were no faces in that file!")
    else:
        #print('---')
        pass
img_list = []
with zipfile.ZipFile('readonly/images.zip', 'r') as files:
    for file in files.infolist():
        # printing out the name of the file
        display(file.filename)
        # name
        name = file.filename
        # image 
        img = Image.open(files.open(name))
        # text
        img_str = pytesseract.image_to_string(img.convert('L'))

        my_dic = {"name":name, "img":img, "text":img_str}
        img_list.append(my_dic)

for element in img_list:
    # test if value is in the text
    if 'Mark' in element["text"]:
        # print out the name of the figure
        print("Results found in file {}".format(element["name"]))

        # index out the faces
        img = element["img"]
        contact_sheet = extract_faces(img,1.35)
        if contact_sheet is not None:
            display(contact_sheet)
        else:
            print("But there were no faces in that file!")
    else:
        #print('---')
        pass

In [ ]:
import zipfile


from PIL import Image
from PIL import ImageDraw
import re
import pytesseract
import cv2 as cv
import numpy as np


# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')


zip_filenames = ['readonly/small_img.zip', 'readonly/images.zip']
contact_image_size = (100, 100)
contact_sheet_width = 5




def get_filenames_from_zipfile(filename):
    with zipfile.ZipFile(filename, 'r') as zip:
        zip.extractall()
        return [info.filename for info in zip.infolist()]


def search_newspaper_for_text(image, search_text):
    return search_text in pytesseract.image_to_string(image)


def search_newspaper_for_faces(image):
    open_cv_image = np.array(image)
    open_cv_image = cv.cvtColor(open_cv_image, cv.COLOR_BGR2RGB)
    open_cv_image = cv.cvtColor(open_cv_image, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(open_cv_image, 1.2, 5)
    return [face for face in faces]


def display_faces(image, faces):
    drawing=ImageDraw.Draw(image)
    for x, y, w, h in faces:
        drawing.rectangle((x, y, x + w, y + h), outline = 'red')
    display(image)
    
def create_contact_sheet(image, faces):
    images = list()
    for x, y, w, h in faces:
        face_image = image.crop((x, y, x + w, y + h))
        face_image.thumbnail(contact_image_size)
        images.append(face_image)
    
    w = contact_image_size[0] * contact_sheet_width
    h = len(faces) // contact_sheet_width
    if len(faces) % contact_sheet_width > 0:
        h += 1
    h *= contact_image_size[1]
    contact_sheet = Image.new(image.mode, (w, h))
    x=0
    y=0
    for img in images:
        contact_sheet.paste(img, (x, y))
        if x + contact_image_size[0] == contact_sheet.width:
            x = 0
            y = y + contact_image_size[1]
        else:
            x = x + contact_image_size[0]
    return contact_sheet




search_texts = ['Christopher', 'Mark']
for zip_filename in zip_filenames:
    filenames = get_filenames_from_zipfile(zip_filename)
    for search_text in search_texts:
        print('Searching with "{}" in "{}"...'.format(search_text, zip_filename))
        for filename in filenames:
            image = Image.open(filename).convert('RGB')
            if search_newspaper_for_text(image, search_text):
                print('Results found in file {}'.format(filename))
                faces = search_newspaper_for_faces(image)
                if len(faces) > 0:
                    display(create_contact_sheet(image, faces))
                else:
                    print('But there were no faces in that file!')